# Nya features
### Kolla vad antal hästar per lopp kan ge

In [108]:
# import modules
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from IPython.display import display

In [109]:
# Läs in all_data.csv
# select numeric columns
df = pd.read_csv('../all_data.csv')
# df.set_index('datum', inplace=True)
df.columns

Index(['datum', 'avd', 'bana', 'häst', 'kusk', 'streck', 'vodds', 'podds',
       'kr', 'spår', 'dist', 'lopp_dist', 'start', 'ålder', 'kön', 'plac',
       'pris', 'h1_dat', 'h1_kusk', 'h1_bana', 'h1_spår', 'h1_plac', 'h1_pris',
       'h1_odds', 'h1_kmtid', 'h2_dat', 'h2_kusk', 'h2_bana', 'h2_spår',
       'h2_plac', 'h2_pris', 'h2_odds', 'h2_kmtid', 'h3_dat', 'h3_kusk',
       'h3_bana', 'h3_spår', 'h3_plac', 'h3_pris', 'h3_odds', 'h3_kmtid',
       'h4_dat', 'h4_kusk', 'h4_bana', 'h4_spår', 'h4_plac', 'h4_pris',
       'h4_odds', 'h4_kmtid', 'h5_dat', 'h5_kusk', 'h5_bana', 'h5_spår',
       'h5_plac', 'h5_pris', 'h5_odds', 'h5_kmtid', 'h1_dist', 'h2_dist',
       'h3_dist', 'h4_dist', 'h5_dist', 'bins', 'h1_auto', 'h2_auto',
       'h3_auto', 'h4_auto', 'h5_auto', 'h1_perf', 'h2_perf', 'h3_perf',
       'h4_perf', 'h5_perf', 'senast', 'delta1', 'delta2', 'delta3', 'delta4',
       'startnr'],
      dtype='object')

In [110]:
# count number per avdeling
df_count = df.groupby(['datum','avd'])['avd'].count()
df['avd_count'] = None
df['avd_count'] = df.groupby(['datum', 'avd'])['avd'].transform('count')

In [111]:
df['avd_count'].unique()
my_remove_list = []
sorted(df['avd_count'].unique())

[4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15]

# Kolla med catboost

### Några hjälp-funktioner

In [112]:
def split_data(df_,train_from_proc=0,test_proc=0.25):
    df=df_.copy()
    alla_datum = df.datum.unique()
    train_from_datum = alla_datum[ int(len(alla_datum)*train_from_proc)]
    print("train from",train_from_datum)
    X_test=None
    y_test=None
    test_from_datum=alla_datum[-1]
    if test_proc:
        selected_data = alla_datum[ alla_datum >= train_from_datum ]
        test_from_datum = selected_data[ int(len(selected_data)*(1-test_proc)) ]
        print("test from",test_from_datum)
        X_test  = df[df.datum >= test_from_datum]
        y_test  = (X_test.plac==1)*1
        X_test  = X_test.drop('plac',axis=1)
        
    X_train = df[(df.datum >= train_from_datum) & (df.datum < test_from_datum) ]
    y_train = (X_train.plac==1)*1
    
    return X_train.drop('plac', axis=1), X_test, y_train, y_test
### Features som inte används vid träning

def remove_features(df_, remove_mer=[]):
    df = df_.drop(['avd', 'startnr', 'vodds', 'podds', 'bins',
                  'h1_dat', 'h2_dat', 'h3_dat', 'h4_dat', 'h5_dat'], axis=1)
    if remove_mer:
        df = df.drop(remove_mer, axis=1)
    return df


### set up for catboost regression

In [119]:
# set up catboost regression model
from catboost import CatBoostRegressor, cv, Pool

X_train, X_test, y_train, y_test = split_data(
    df, train_from_proc=0.3, test_proc=0.25)
X_train = remove_features(X_train, remove_mer=my_remove_list )
X_test = remove_features(X_test, remove_mer=my_remove_list )

cat_features = list(X_train.select_dtypes('object').columns)
X_train[cat_features] = X_train[cat_features].fillna('missing')
X_test[cat_features] = X_test[cat_features].fillna('missing')
print(cat_features)
pool = Pool(X_train, y_train, cat_features=cat_features)
test_pool = Pool(X_test, y_test, cat_features=cat_features)
X_train.shape, X_test.shape


train from 2017-02-26
test from 2020-07-05
['datum']


((22608, 7), (7657, 7))

In [120]:
# run catboost
cb = CatBoostRegressor(
    iterations=1000,
    eval_metric='MAE',
    early_stopping_rounds=100,
    random_seed=42,
    verbose=100
)
cb.fit(pool, eval_set=test_pool)


Learning rate set to 0.083084
0:	learn: 0.1562140	test: 0.1559061	best: 0.1559061 (0)	total: 79.8ms	remaining: 1m 19s
100:	learn: 0.1309000	test: 0.1350799	best: 0.1332685 (67)	total: 3.9s	remaining: 34.7s
Stopped by overfitting detector  (100 iterations wait)

bestTest = 0.1332685133
bestIteration = 67

Shrink model to first 68 iterations.


Med streck:  
alla:  
bestTest = 0.1338137229  
bestIteration = 210  

alla 0:or borta:  
bestTest = 0.1332685133  
bestIteration = 67  

Utan streck:  
alla:  
bestTest = 0.1593303093  
bestIteration = 0 

alla 0:or borta:  
bestTest = 0.137245835  
bestIteration = 113  






In [121]:
my_feature_imp =cb.get_feature_importance(prettified=True)
my_remove_list2 = my_feature_imp[my_feature_imp.Importances <= 0]['Feature Id'].to_list()
print(my_remove_list2)
my_remove_list=my_remove_list+my_remove_list2
# remove duplicates
my_remove_list = list(set(my_remove_list))


[]


In [122]:
my_feature_imp

,Feature Id,Importances
0,streck,62.382651
1,datum,15.348318
2,h2_odds,5.557340
3,h5_perf,5.268639
4,h1_perf,4.337421
5,h3_odds,3.843078
6,h2_perf,3.262552
